In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns;sns.set()
%matplotlib inline


In [2]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from smotefuna import Smotfuna

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler

In [18]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [ ]:
#optional
from sklearn.decomposition import PCA
from sklearn import model_selection


In [4]:
from prepare import prepare_data
df=pd.read_csv(r'C:\Users\ELTE IK EFOP\Desktop\myresearch\edm\oulad_data_2403.csv').set_index('id_student')


In [5]:
df.head(2)

,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,...,oucontent,ouelluminate,ouwiki,page,resource,subpage,url,CMA,Exam,TMA
id_student,,,,,,,,,,,,,,,,,,,,,
24213,F,East Anglian Region,A Level or Equivalent,40-50%,0-35,0,90,N,Withdrawn,-128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40419,M,East Midlands Region,Lower Than A Level,20-30%,35-55,0,60,N,Withdrawn,-46.0,...,9.0,6.0,7.0,2.0,3.0,20.0,11.0,1.34,0.0,5.7


In [6]:
def make_Binary(x):
    if (x=='Withdrawn'):
        return 1
    else:
        return 0
    
def prepare_data(df,ml=241,remove_before=False,drop_reg=False,remove_demo=True,remove_some=[]):# ml refers to module length
    df2=df.copy()
    cols=['externalquiz','forumng','glossary','homepage','oucontent','ouelluminate','ouwiki','page','resource','subpage','url','Exam']
    cols2=['CMA','TMA']
    reg=['date_unregistration','date_registration']
    target=['final_result']
    df2[cols]=df2[cols].fillna(0)
    df2[cols]=df2[cols].apply(pd.to_numeric,downcast='integer')
    df2[cols2]=df2[cols2].fillna(0).round(3)
    df2['date_unregistration']=df2['date_unregistration'].replace(np.nan,ml)
    df2['final_result'] = df2['final_result'].apply(make_Binary)
    df2['final_result']=df2['final_result'].apply(pd.to_numeric,downcast='integer')
    #df2=df2.loc[df2['date_registration'].notna()]# remove records with nan
    if remove_before:
        df2=df2[df2['date_unregistration']>0]
    if  remove_demo:
        final_cols=cols+cols2+target
        if remove_some:
            final_cols=[c for c in final_cols if c not in remove_some]
        return df2[final_cols]
    else:
        
        if drop_reg:
            df2=df2.drop(reg,axis=1)
            reg=[]
        else:
            df2[reg]=df2[reg].apply(pd.to_numeric,downcast='integer')
        imd_band_mapping={'0-10%':1,'10-20':2,'20-30%':3,'30-40%':4,'40-50%':5,'50-60%':6,'60-70%':7,'70-80%':8,'80-90%':9,'90-100%':10}
        he_mapping={'A Level or Equivalent':3,'Lower Than A Level':2,'HE Qualification':4,'Post Graduate Qualification':5,'No Formal quals':1}
        age_mapping={'0-35':1,'35-55':2,'55<=':3}
        df2['imd_band']=df2['imd_band'].map(imd_band_mapping).apply(pd.to_numeric,downcast='integer')
        df2['highest_education']=df2['highest_education'].map(he_mapping).apply(pd.to_numeric,downcast='integer')
        df2['age_band']=df2['age_band'].map(age_mapping).apply(pd.to_numeric,downcast='integer')
        #cols3=['imd_band','highest_education','age_band']
        #cols4=['num_of_prev_attempts','studied_credits']
        df2=pd.get_dummies(df2,columns=['gender','region','disability'])
        colss=[col for col in df2.columns if col not in cols+cols2+reg]
        df2[colss]=df2[colss].apply(pd.to_numeric,downcast='integer')
        all_cols=df2.columns.tolist()# arrange columns so target, final_reslut at the end
        all_cols.remove('final_result')
        all_cols.append('final_result')
        # arrnge column so label in teh end
 
    return df2[all_cols]

In [7]:
def apply_missForest(df,label='final_result'):
    X=df.copy()
    y=X.pop(label)
    imputer = MissForest()
    X_imputed = imputer.fit_transform(X) # do not need label
    D1c=pd.DataFrame(X_imputed,columns=X.columns,index=df.index)
    D1c[label]=df[label]
    return D1c

In [8]:
def apply_smotefuba(data,label='final_result',return_X_y=True): # the label should be last column so I considered that in last steps
    smtf=Smotfuna(data)
    s,mn,mj,f=smtf.balance()
    data_balanced= pd.DataFrame(f,columns=data.columns)
    #D1c_balanced.shape,D1c.shape
    data_balanced[label].loc[(data_balanced[label] ==2)] = 1 # the algorithm uses 1 and 2 but in datest we have 0 and 1
    #D1c_balanced['label'].value_counts()
    if return_X_y:
        cols_num=len(data.columns)-1
        X_somtfuna=data_balanced.iloc[:,:cols_num]
        y_somtfuna=data_balanced.iloc[:,cols_num]
        return X_somtfuna,y_somtfuna
    return data_balanced

In [16]:
def apply(X,y,cv=10,std=False):
    scoring = ['accuracy','precision', 'recall', 'f1']
    models = []
    models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=100)))
    #models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('CART', DecisionTreeClassifier(random_state = 42)))
    models.append(('NB', GaussianNB()))
    models.append(('SVM', SVC(random_state = 42)))
    models.append(('RF', RandomForestClassifier(random_state = 42)))
    models.append(('XGB', XGBClassifier(use_label_encoder=False,verbosity=0,random_state = 42)))
    need_scale=['SVM','LR','LDA']
#     if std:
#         scaler = StandardScaler()
#         X=scaler.fit_transform(X)
    scaler = StandardScaler()
    X_scaled=scaler.fit_transform(X)
    results={}
    for name, model in models:
        subscor={}
        if name in need_scale:
            scores = cross_validate(model, X_scaled, y,cv=cv, scoring=scoring)
        else:
            scores = cross_validate(model, X, y,cv=cv, scoring=scoring)
        for metric in scoring:
            subscor[metric]=round(scores['test_'+metric].mean(),3)
        results[name]=subscor
    res_df=pd.DataFrame.from_dict(results,orient='index')
    return res_df

In [15]:
del apply

NameError: name 'apply' is not defined

In [57]:
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import shap

In [56]:
del train_test_split,StratifiedKFold,shap

NameError: name 'train_test_split' is not defined

In [74]:
def shap_cv(X,y,ns=5,skf=True):
    

    #loading and preparing the data
#     iris = load_breast_cancer()
#     X = iris.data
#     y = iris.target
#     columns = iris.feature_names
    columns=X.columns
    #if you don't shuffle you wont need to keep track of test_index, but I think 
    #it is always good practice to shuffle your data
#     if skf:
#         kf = StratifiedKFold(n_splits=ns,shuffle=False)
#         splits=kf.split(X,y)
#     else:
#         kf = KFold(n_splits=ns,shuffle=False)
#         splits=kf.split(X)
#     for train_index, test_index in splits:
    kf=KFold(n_splits=ns,shuffle=False)
    list_shap_values = list()
    list_test_sets = list()
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train = pd.DataFrame(X_train,columns=columns)
        X_test = pd.DataFrame(X_test,columns=columns)

        #training model
        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)

        #explaining model
        explainer = shap.TreeExplainer(clf)
        shap_values = explainer.shap_values(X_test)
        #for each iteration we save the test_set index and the shap_values
        list_shap_values.append(shap_values)
        list_test_sets.append(test_index)



    # flatten list of lists, pick the sv for 1 class, stack the result
    shap_values = np.vstack([sv[1] for sv in list_shap_values])
    sv = np.abs(shap_values).mean(0)  # <-- error corrected    
    importance_df = pd.DataFrame({
        "column_name": columns,
        "shap_values": sv
    })
    
    return importance_df.sort_values(by='shap_values',ascending=False)


In [72]:
del shap_cv

In [45]:
def importance(X,y,cv=5,scoring='accuracy'):
    importance={}
    clf=RandomForestClassifier(random_state = 42)#n_estimators =10, random_state = 42, class_weight="balanced"
    output = cross_validate(clf, X, y, cv=cv, scoring = scoring, return_estimator =True)
    for idx,estimator in enumerate(output['estimator']):
        print("Features sorted by their score for estimator {}:".format(idx))
        feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = X.columns,
                                        columns=['importance']).sort_values('importance', ascending=False)
        importance[idx]=feature_importances
    #return feature_importances
    return importance

In [ ]:
def importance_svm(,cv=5,scoring='accuracy'):
    importance={}
    clf=SVC(random_state = 42)#n_estimators =10, random_state = 42, class_weight="balanced"
    output = cross_validate(clf, X, y, cv=cv, scoring = scoring, return_estimator =True)
    for idx,estimator in enumerate(output['estimator']):
        print("Features sorted by their score for estimator {}:".format(idx))
        feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = X.columns,
                                        columns=['importance']).sort_values('importance', ascending=False)
        importance[idx]=feature_importances


In [ ]:
from sklearn.inspection import permutation_importance
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

svc =  SVC(kernel='rbf', C=2)
svc.fit(X_train, y_train)

perm_importance = permutation_importance(svc, X_test, y_test)

feature_names = ['feature1', 'feature2', 'feature3', ...... ]
features = np.array(feature_names)

sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")


In [44]:
del importance

In [9]:
df2=prepare_data(df,ml=241,remove_before=True,drop_reg=True,remove_demo=True,remove_some=['glossary','page','Exam'])


In [10]:
df2.head(2)

,externalquiz,forumng,homepage,oucontent,ouelluminate,ouwiki,resource,subpage,url,CMA,TMA,final_result
id_student,,,,,,,,,,,,
40419,1,56,29,9,6,7,3,20,11,1.34,5.700,1
41060,15,28,80,116,5,13,24,124,6,16.37,47.225,0


In [80]:
X,y=df2.drop('final_result',1),df2['final_result']

In [31]:
non_bal_res=apply(X,y,cv=5)
non_bal_res

,accuracy,precision,recall,f1
LR,0.777,0.607,0.662,0.633
CART,0.758,0.576,0.617,0.595
NB,0.740,0.532,0.885,0.664
SVM,0.799,0.617,0.808,0.700
RF,0.778,0.613,0.665,0.636
XGB,0.772,0.599,0.656,0.626


In [21]:
sf_X,sf_y=apply_smotefuba(df2)

In [29]:
bal_res=apply(sf_X,sf_y,cv=5)

In [30]:
bal_res

,accuracy,precision,recall,f1
LR,0.738,0.728,0.760,0.743
CART,0.741,0.746,0.712,0.720
NB,0.567,0.557,0.585,0.569
SVM,0.728,0.742,0.669,0.662
RF,0.787,0.791,0.777,0.769
XGB,0.780,0.781,0.769,0.759


In [46]:
rf_b_import=importance(sf_X,sf_y,cv=5)

Features sorted by their score for estimator 0:
Features sorted by their score for estimator 1:
Features sorted by their score for estimator 2:
Features sorted by their score for estimator 3:
Features sorted by their score for estimator 4:


In [47]:
rf_b_import

{0:               importance
 TMA             0.230525
 CMA             0.152987
 url             0.092583
 forumng         0.083784
 oucontent       0.072933
 homepage        0.071770
 resource        0.069754
 ouelluminate    0.064633
 subpage         0.063917
 externalquiz    0.049903
 ouwiki          0.047210,
 1:               importance
 TMA             0.224160
 CMA             0.148721
 url             0.098091
 forumng         0.090434
 homepage        0.074150
 oucontent       0.073245
 resource        0.069026
 subpage         0.066135
 ouelluminate    0.062351
 externalquiz    0.050201
 ouwiki          0.043485,
 2:               importance
 TMA             0.225770
 CMA             0.152409
 forumng         0.098186
 url             0.088861
 subpage         0.072288
 homepage        0.070628
 resource        0.069480
 oucontent       0.067262
 ouelluminate    0.063217
 externalquiz    0.050806
 ouwiki          0.041093,
 3:               importance
 TMA             0.1968

In [25]:
import SHAP as shap

ModuleNotFoundError: No module named 'SHAP'

In [75]:
rf_shap=shap_cv(sf_X,sf_y,skf=False)

In [76]:
rf_shap

,column_name,shap_values
10,TMA,0.143948
9,CMA,0.086233
1,forumng,0.062204
8,url,0.060197
4,ouelluminate,0.053809
2,homepage,0.024448
3,oucontent,0.024149
6,resource,0.021826
7,subpage,0.019318
0,externalquiz,0.017479


In [79]:
def shap_cv_svm(X,y,ns=5,skf=False):
    

    columns=X.columns
    if skf:
        kf = StratifiedKFold(n_splits=ns,shuffle=False)
        splits=kf.split(X,y)
    else:
        kf = KFold(n_splits=ns,shuffle=False)
        splits=kf.split(X)
    list_shap_values = list()
    list_test_sets = list()
    for train_index, test_index in splits:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train = pd.DataFrame(X_train,columns=columns)
        X_test = pd.DataFrame(X_test,columns=columns)

        #training model
#         clf = RandomForestClassifier(random_state=42)
#         clf.fit(X_train, y_train)
        clf = SVC(kernel='rbf', probability=True)
        clf.fit(X_train, y_train)
        #print_accuracy(svc_linear.predict)

        # explain all the predictions in the test set
        explainer = shap.KernelExplainer(clf.predict_proba, X_train)
        shap_values = explainer.shap_values(X_test)
        #shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)

        #explaining model
#         explainer = shap.TreeExplainer(clf)
#         shap_values = explainer.shap_values(X_test)
        #for each iteration we save the test_set index and the shap_values
        list_shap_values.append(shap_values)
        list_test_sets.append(test_index)



    # flatten list of lists, pick the sv for 1 class, stack the result
    shap_values = np.vstack([sv[1] for sv in list_shap_values])
    sv = np.abs(shap_values).mean(0)  # <-- error corrected    
    importance_df = pd.DataFrame({
        "column_name": columns,
        "shap_values": sv
    })
    
    return importance_df.sort_values(by='shap_values',ascending=False)


In [ ]:
svm_shap=shap_cv_svm(X,y,skf=True)
svm_shap